<a href="https://colab.research.google.com/github/ipeirotis/sql_autograding/blob/main/Zero_Few_shot_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install --upgrade openai
!pip install -q google-cloud-secret-manager
!pip3 install -U -q PyMySQL sqlalchemy sql_magic

In [2]:
from google.colab import auth

# Login using the account that has access to the Google project
# in order to access the resources for the project
auth.authenticate_user()

In [3]:
from google.cloud import secretmanager
import os

def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload of the given secret version and return it.

    Args:
        project_id (str): Google Cloud project ID.
        secret_id (str): ID of the secret to access.
        version_id (str): ID of the version to access.
    Returns:
        str: The secret version's payload, or None if
        the version does not exist.
    """
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")


openai_key = access_secret_version("sql-autograding", "openai-gpt4-32k", "latest")

In [4]:
import openai
openai.api_key = openai_key

In [5]:
from google.colab import files
import io
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
import gcsfs
# load the cleaned dataset
fs = gcsfs.GCSFileSystem(project="sql_autograding")
with fs.open("gs://sql_autograding/clean_submissions_final.csv") as f:
    data = pd.read_csv(f)

In [7]:
data = data.drop(columns="Unnamed: 0")
data.head()

,QuestionID,SubmittedAnswer,StudentID,Score
0,50,"SELECT A.CustomerID, A.CompanyName,...",ab3864,10.0
1,54,"SELECT O.CustomerID, SUM(D.Quantity * D....",ab3864,6.5
2,55,"Select O.OrderID, O.CustomerID, Count(D.Produ...",ab3864,5.5
3,61,"use northwind; SELECT ShipCountry, AVG(...",ab3864,7.0
4,56,"Select C.CompanyName, C.CustomerID, sum(case w...",ab3864,10.0


In [8]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [9]:
with fs.open("gs://sql_autograding/Database Homeworks.csv") as f:
    hw = pd.read_csv(f)
hw.head()

,QuestionID,HomeWork,Question,Database,AnswerKey
0,0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...
1,1,Assignment 2B: Selection Queries,List the ten shortest tracks in terms of playi...,music,SELECT * FROM track ORDER BY time LIMIT 10
2,2,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR..."
3,3,Assignment 2B: Selection Queries,Show all the albums.,music,SELECT * FROM album
4,4,Assignment 2B: Selection Queries,Show all the artists.,music,SELECT * FROM artist


In [10]:
df_all = pd.merge(data, hw, how='left', on='QuestionID')
df_all.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey
0,50,"SELECT A.CustomerID, A.CompanyName,...",ab3864,10.0,Final Exam,Hard question. Requires use of subqueries and ...,northwind,"WITH total_spent AS ( SELECT C.CustomerID, C.C..."
1,54,"SELECT O.CustomerID, SUM(D.Quantity * D....",ab3864,6.5,Final Exam,"For each customer, we want to calculate how mu...",northwind,"SELECT C.CustomerID, C.CompanyName , COALESCE(..."
2,55,"Select O.OrderID, O.CustomerID, Count(D.Produ...",ab3864,5.5,Final Exam,"For each order, show the customerID who placed...",northwind,"SELECT O.OrderID, O.CustomerID , SUM(D.Quantit..."
3,61,"use northwind; SELECT ShipCountry, AVG(...",ab3864,7.0,Final Exam,We want to identify the ship countries with th...,northwind,"SELECT ShipCountry, ROUND(AVG(Freight),2) AS A..."
4,56,"Select C.CompanyName, C.CustomerID, sum(case w...",ab3864,10.0,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order..."


## Further preprocessing

In [11]:
db_list = df_all['Database'].unique().tolist()
print(db_list)


def db_schema(db_name):
    """
    Connects to a database and returns the schema of each table in the database.

    The function connects to a specific database using SQLAlchemy. It then retrieves the list of tables in
    the database and for each table, it queries the schema (i.e., the list of fields/columns) and stores it
    in a dictionary. The function returns a list of such dictionaries, with each dictionary representing a table
    and its corresponding schema.

    Args:
        db_name (str): The name of the database to connect to and retrieve schemas from.

    Returns:
        list: A list of dictionaries, with each dictionary containing the name of a table as the key and a
              list of its fields as the value.

    Example:
        >>> db_schema('flights')
        {'m_airports': ['airport', 'state', 'state_name']},
        {'m_ticket_prices': ['origin', 'dest', 'carrier', 'fare', 'fare_per_mile','passengers', 'distance']
        ...
        },
    """

    student_password = access_secret_version(
        "sql-autograding", "db_student_password", "1"
    )

    conn_string = (
        "mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4".format(
            host="db.ipeirotis.org",
            user="student",
            password=student_password,
            db=db_name,
            encoding="utf8mb4",
        )
    )
    engine = create_engine(conn_string)

    with engine.begin() as conn:
        tables = pd.read_sql_query(sql=text("show tables"), con=conn)
        tables = tables.iloc[:, -1].tolist()

        schema_list = []
        for t in tables:
            d = pd.read_sql_query(sql=text(f"describe {t}"), con=conn)
            table_schema = {t: d.loc[:, "Field"].tolist()}
            schema_list.append(table_schema)
    return schema_list

['northwind', 'facebook', 'restaurants', 'imdb', 'music', 'flights', 'collisions']


In [12]:
# generates the schema for each of the databases in the dataset
flights_schema = db_schema("flights")
imdb_schema = db_schema("imdb")
music_schema = db_schema("music")
restaurants_schema = db_schema("restaurants")
facebook_schema = db_schema("facebook")
northwind_schema = db_schema("northwind")
collisions_schema = db_schema("collisions")

print(facebook_schema)

[{'Concentration': ['ProfileID', 'Concentration']}, {'FavoriteBooks': ['ProfileID', 'Book']}, {'FavoriteMovies': ['ProfileID', 'Movie']}, {'FavoriteMusic': ['ProfileID', 'Music']}, {'FavoriteTVShows': ['ProfileID', 'TVShow']}, {'Hobbies': ['ProfileID', 'Hobby']}, {'LookingFor': ['ProfileID', 'LookingFor']}, {'Orientation': ['ProfileID', 'InterestedIn']}, {'Profiles': ['ProfileID', 'Name', 'MemberSince', 'LastUpdate', 'School', 'Status', 'Sex', 'Birthday', 'AIM', 'Website', 'PoliticalViews', 'Geography', 'HighSchool', 'HomeTown', 'HomeState', 'Residence', 'CurrentAddress', 'CurrentTown', 'CurrentState']}, {'Relationship': ['ProfileID', 'Status']}, {'chart': ['music', 'gender', 'cnt', 'music_rank']}, {'popular_music': ['music', 'cnt']}, {'popular_music_by_sex': ['music', 'gender', 'cnt']}]


In [13]:
# creates a dictionary that maps each schema to each database and add the schema as a new column in the dataframe
schema_mapping = {
    "flights": flights_schema,
    "imdb": imdb_schema,
    "music": music_schema,
    "restaurants": restaurants_schema,
    "facebook": facebook_schema,
    "northwind": northwind_schema,
    "collisions": collisions_schema,
}

df_all["schema"] = df_all["Database"].map(schema_mapping)
df_all.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema
0,50,"SELECT A.CustomerID, A.CompanyName,...",ab3864,10.0,Final Exam,Hard question. Requires use of subqueries and ...,northwind,"WITH total_spent AS ( SELECT C.CustomerID, C.C...","[{'Categories': ['CategoryID', 'CategoryName',..."
1,54,"SELECT O.CustomerID, SUM(D.Quantity * D....",ab3864,6.5,Final Exam,"For each customer, we want to calculate how mu...",northwind,"SELECT C.CustomerID, C.CompanyName , COALESCE(...","[{'Categories': ['CategoryID', 'CategoryName',..."
2,55,"Select O.OrderID, O.CustomerID, Count(D.Produ...",ab3864,5.5,Final Exam,"For each order, show the customerID who placed...",northwind,"SELECT O.OrderID, O.CustomerID , SUM(D.Quantit...","[{'Categories': ['CategoryID', 'CategoryName',..."
3,61,"use northwind; SELECT ShipCountry, AVG(...",ab3864,7.0,Final Exam,We want to identify the ship countries with th...,northwind,"SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...","[{'Categories': ['CategoryID', 'CategoryName',..."
4,56,"Select C.CompanyName, C.CustomerID, sum(case w...",ab3864,10.0,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","[{'Categories': ['CategoryID', 'CategoryName',..."


In [14]:
df_all['full_score'] = df_all.groupby('QuestionID')['Score'].transform('max')
df_all.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score
0,50,"SELECT A.CustomerID, A.CompanyName,...",ab3864,10.0,Final Exam,Hard question. Requires use of subqueries and ...,northwind,"WITH total_spent AS ( SELECT C.CustomerID, C.C...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0
1,54,"SELECT O.CustomerID, SUM(D.Quantity * D....",ab3864,6.5,Final Exam,"For each customer, we want to calculate how mu...",northwind,"SELECT C.CustomerID, C.CompanyName , COALESCE(...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0
2,55,"Select O.OrderID, O.CustomerID, Count(D.Produ...",ab3864,5.5,Final Exam,"For each order, show the customerID who placed...",northwind,"SELECT O.OrderID, O.CustomerID , SUM(D.Quantit...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0
3,61,"use northwind; SELECT ShipCountry, AVG(...",ab3864,7.0,Final Exam,We want to identify the ship countries with th...,northwind,"SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...","[{'Categories': ['CategoryID', 'CategoryName',...",7.5
4,56,"Select C.CompanyName, C.CustomerID, sum(case w...",ab3864,10.0,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0


## Generating Mega Prompt
===============================

a. We are working with the XXX database, which has the following tables: artist(id, first_name, last_name) album(id, artist_id, name) track....

b. The request to the student is "Fetch all the tracks for user X"

c. The model answer is ..... (note that we may have multiple correct answers, the model answer is just an example)

d. The student answer was .....

e. Previously, students have submitted these answers and got back these responses and grades:

e1. submission: ..... , grade: ..... , feedback

e2. submission: ..... , grade: ..... , feedback

e3. submission: ..... , grade: ..... , feedback

Please provide a grade and feedback for the student

===============================

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
from typing import List

def generate_submissions_string(
    train_df: pd.DataFrame, submission_size: int = None
) -> str:
    """
    Generate a string that concatenates student submissions up to the given size.

    Args:
        train_df (pd.DataFrame): The DataFrame containing the training data.
        submission_size (int, optional): The number of submissions to include in the string. If None, include all submissions.

    Returns:
        str: A string containing the concatenated submissions.
    """
    submissions = ""
    for i, row in train_df.iterrows():
        if submission_size is not None and i >= submission_size:
            break
        submission = row["SubmittedAnswer"]
        grade = row["Score"]
        full_grade = row["full_score"]
        submissions += f"e{i}. submission: {submission}, grade: {grade}/{full_grade}\\n\\n"
    return submissions


def generate_mega_prompt_for_id(
    df: pd.DataFrame, id: str, submission_size: int = None
) -> str:
    """
    Generate a mega prompt for a specific QuestionId.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        id (str): The QuestionId to generate the mega prompt for.
        submission_size (int, optional): The number of submissions to include in the mega prompt. If None, include all submissions.

    Returns:
        str: The mega prompt.
    """
    temp = df[df["QuestionID"] == id]

    # If there's only one row or less, return None or handle it in a special way
    if len(temp) == 0:
      return None
    if len(temp) == 1:
      # return the only row as test answer
      temp = temp.reset_index()
      submissions = ''
      db = temp.loc[0, "Database"]
      tables = temp.loc[0, "schema"]
      question = temp.loc[0, "Question"]
      key = temp.loc[0, "AnswerKey"]
      full_score = temp.loc[0, "full_score"]
      test = temp.loc[0, "SubmittedAnswer"]

      mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade (out of 10 points) and feedback for the student
        """
      return mega_p

    # making test size 1
    train_df, test_df = train_test_split(temp, test_size=1, random_state=1234)
    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    submissions = generate_submissions_string(train_df, submission_size)

    db = train_df.loc[0, "Database"]
    tables = train_df.loc[0, "schema"]
    question = train_df.loc[0, "Question"]
    key = train_df.loc[0, "AnswerKey"]
    full_score = temp.loc[0, "full_score"]
    test = test_df.loc[0, "SubmittedAnswer"]

    mega_p = f"""
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

      Please provide a grade out of the full score and feedback for the student
      """
    return mega_p


def generate_mega_prompts(
    df: pd.DataFrame, submission_size: int = None
) -> pd.DataFrame:
    """
    Generate a DataFrame of mega prompts for each unique QuestionId in the given DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        submission_size (int, optional): The number of submissions to include in each mega prompt. If None, include all submissions.

    Returns:
        pd.DataFrame: A DataFrame containing the mega prompts.
    """
    result = []
    qids = df["QuestionID"].unique()

    for id in qids:
        mega_p = generate_mega_prompt_for_id(df, id, submission_size)
        new_row = {"QuestionID": id, "MegaPrompt": mega_p}
        result.append(new_row)

    mega_df = pd.DataFrame(result)

    return mega_df

In [17]:
from openai import OpenAI

client = OpenAI(
  api_key=openai_key,  # this is also the default, it can be omitted
)

In [18]:
# Answering Mega Prompt Using GPT 3.5 model
# Deal with rate limit of GPT 3.5-turbo model
from tenacity import(retry, stop_after_attempt, wait_random_exponential)
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT3_generation_with_backoff(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response.choices[0].message.content

In [19]:
# Answering Mega Prompt Using GPT 4 model
# dealing with rate limit of GPT4
import time
# further control rate limit for GPT-4
rate_limit_per_minute = 40000
delay = 60.0 / rate_limit_per_minute

def delayed_completion(p):
  time.sleep(delay)
  return GPT4_generation_with_backoff(p)

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT4_generation_with_backoff(prompt):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response.choices[0].message.content

## DataFrame with Zero-Shot Prompts

In [20]:
from tqdm import tqdm
import numpy as np

data1, _ = train_test_split(df_all, train_size=800, stratify=df_all['QuestionID'], random_state=1234)
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score
4777,7,USE music; SELECT* FROM album Where album_n...,qz2313,1.0,Assignment 2C: Filtering Queries,Find all albums with a title that begins with ...,music,SELECT * FROM album WHERE album_name >= 'F' AN...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
6377,74,SELECT * FROM Profiles WHERE Name LIKE 'P%' ...,aso6907,0.0,Module 2 Practice: Filtering queries,Find all students with first names starting wi...,facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'P% I%',"[{'Concentration': ['ProfileID', 'Concentratio...",0.0
1949,90,"select firstN,lastN,affiliation from Critic w...",ss13871,0.0,Module 2 Practice: Filtering queries,Show the names of all the food critics who wor...,restaurants,SELECT * FROM Critic WHERE affiliation='NYT' O...,"[{'Critic': ['cID', 'firstN', 'lastN', 'affili...",0.0
8869,26,"SELECT TP.user_id \t, COUNT(TP.track_id) AS T...",re2272,0.7,Assignment 4: Aggregations,"For every user, show the total tracks they pla...",music,"SELECT U.user_id, COUNT(T.track_id) AS tracks_...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0
8016,0,SELECT artist_name FROM artist ORDER BY arti...,cas760,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0


In [21]:
# A function that parses grade and feedback from GPT responses
def parse_grade_feedback(text):
  colon_pos = text.find(":")
  divider_pos = text.find("/")

  grade = text[colon_pos+2:divider_pos]

  feedback_start = text.find("Feedback: ")
  if feedback_start != -1:
      feedback = text[feedback_start + len("Feedback: "):]
  else:
      feedback = None
  return grade, feedback

In [22]:
lst = []
for i,r in data1.iterrows():
  # generate zero-shot mega prompt for each row of the dataframe
  lst.append(generate_mega_prompts(r.to_frame().T,0).loc[0,'MegaPrompt'])
# append mega prompts as a new column to the dataframe
data1['0_mp'] = lst
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,0_mp
4777,7,USE music; SELECT* FROM album Where album_n...,qz2313,1.0,Assignment 2C: Filtering Queries,Find all albums with a title that begins with ...,music,SELECT * FROM album WHERE album_name >= 'F' AN...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...
6377,74,SELECT * FROM Profiles WHERE Name LIKE 'P%' ...,aso6907,0.0,Module 2 Practice: Filtering queries,Find all students with first names starting wi...,facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'P% I%',"[{'Concentration': ['ProfileID', 'Concentratio...",0.0,\n a. We are working with the facebook ...
1949,90,"select firstN,lastN,affiliation from Critic w...",ss13871,0.0,Module 2 Practice: Filtering queries,Show the names of all the food critics who wor...,restaurants,SELECT * FROM Critic WHERE affiliation='NYT' O...,"[{'Critic': ['cID', 'firstN', 'lastN', 'affili...",0.0,\n a. We are working with the restauran...
8869,26,"SELECT TP.user_id \t, COUNT(TP.track_id) AS T...",re2272,0.7,Assignment 4: Aggregations,"For every user, show the total tracks they pla...",music,"SELECT U.user_id, COUNT(T.track_id) AS tracks_...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...
8016,0,SELECT artist_name FROM artist ORDER BY arti...,cas760,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...


In [23]:
tqdm.pandas()
# generate GPT4 responses for each row and parse grade and feedback as new columns
data1['4_0_answer'] = data1['0_mp'].progress_apply(GPT4_generation_with_backoff)
data1['4_0_grade'], data1['4_0_feedback'] = zip(*data1['4_0_answer'].apply(parse_grade_feedback))
data1.head()

100%|██████████| 800/800 [1:11:31<00:00,  5.36s/it]


,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,0_mp,4_0_answer,4_0_grade,4_0_feedback
4777,7,USE music; SELECT* FROM album Where album_n...,qz2313,1.0,Assignment 2C: Filtering Queries,Find all albums with a title that begins with ...,music,SELECT * FROM album WHERE album_name >= 'F' AN...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,Grade: 9/10\n\nFeedback: Your answer is almost...,9,Your answer is almost correct and you have und...
6377,74,SELECT * FROM Profiles WHERE Name LIKE 'P%' ...,aso6907,0.0,Module 2 Practice: Filtering queries,Find all students with first names starting wi...,facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'P% I%',"[{'Concentration': ['ProfileID', 'Concentratio...",0.0,\n a. We are working with the facebook ...,Grade: 2/10\n\nFeedback: Your SQL query is inc...,2,Your SQL query is incorrect. The WHERE clause ...
1949,90,"select firstN,lastN,affiliation from Critic w...",ss13871,0.0,Module 2 Practice: Filtering queries,Show the names of all the food critics who wor...,restaurants,SELECT * FROM Critic WHERE affiliation='NYT' O...,"[{'Critic': ['cID', 'firstN', 'lastN', 'affili...",0.0,\n a. We are working with the restauran...,Grade: 10/10\n\nFeedback: Great job! Your quer...,10,Great job! Your query correctly retrieves the ...
8869,26,"SELECT TP.user_id \t, COUNT(TP.track_id) AS T...",re2272,0.7,Assignment 4: Aggregations,"For every user, show the total tracks they pla...",music,"SELECT U.user_id, COUNT(T.track_id) AS tracks_...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,Grade: 8/10\n\nFeedback: Your SQL query is mos...,8,"Your SQL query is mostly correct, but you miss..."
8016,0,SELECT artist_name FROM artist ORDER BY arti...,cas760,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10,Great job! Your SQL query is correct and it su...


In [24]:
data1.to_csv('zero_shot_4.csv')

In [ ]:
fs = gcsfs.GCSFileSystem(project="sql_autograding")
with fs.open("gs://sql_autograding/zero_shot_4.csv") as f:
    data1 = pd.read_csv(f)

In [25]:
# generate GPT3.5-turbo responses for each row and parse grade and feedback as new columns
tqdm.pandas()
data1['3_0_answer'] = data1['0_mp'].progress_apply(GPT3_generation_with_backoff)
data1['3_0_grade'], data1['3_0_feedback'] = zip(*data1['3_0_answer'].apply(parse_grade_feedback))
data1.head()

100%|██████████| 800/800 [1:03:01<00:00,  4.73s/it]


,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,0_mp,4_0_answer,4_0_grade,4_0_feedback,3_0_answer,3_0_grade,3_0_feedback
4777,7,USE music; SELECT* FROM album Where album_n...,qz2313,1.0,Assignment 2C: Filtering Queries,Find all albums with a title that begins with ...,music,SELECT * FROM album WHERE album_name >= 'F' AN...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,Grade: 9/10\n\nFeedback: Your answer is almost...,9,Your answer is almost correct and you have und...,Grade: 7/10\n\nFeedback: \n- The student corre...,7,\n- The student correctly identified the corre...
6377,74,SELECT * FROM Profiles WHERE Name LIKE 'P%' ...,aso6907,0.0,Module 2 Practice: Filtering queries,Find all students with first names starting wi...,facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'P% I%',"[{'Concentration': ['ProfileID', 'Concentratio...",0.0,\n a. We are working with the facebook ...,Grade: 2/10\n\nFeedback: Your SQL query is inc...,2,Your SQL query is incorrect. The WHERE clause ...,Grade: 5/10\n\nFeedback: \n- The student's que...,5,\n- The student's query is close to the correc...
1949,90,"select firstN,lastN,affiliation from Critic w...",ss13871,0.0,Module 2 Practice: Filtering queries,Show the names of all the food critics who wor...,restaurants,SELECT * FROM Critic WHERE affiliation='NYT' O...,"[{'Critic': ['cID', 'firstN', 'lastN', 'affili...",0.0,\n a. We are working with the restauran...,Grade: 10/10\n\nFeedback: Great job! Your quer...,10,Great job! Your query correctly retrieves the ...,Grade: 7/10\n\nFeedback: \n- The student corre...,7,\n- The student correctly identified the table...
8869,26,"SELECT TP.user_id \t, COUNT(TP.track_id) AS T...",re2272,0.7,Assignment 4: Aggregations,"For every user, show the total tracks they pla...",music,"SELECT U.user_id, COUNT(T.track_id) AS tracks_...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,Grade: 8/10\n\nFeedback: Your SQL query is mos...,8,"Your SQL query is mostly correct, but you miss...",Grade: 8/10\n\nFeedback: \n- The student's que...,8,\n- The student's query correctly joins the ta...
8016,0,SELECT artist_name FROM artist ORDER BY arti...,cas760,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10,Great job! Your SQL query is correct and it su...,Grade: 9.5/10\n\nFeedback: Great job! Your que...,9.5,Great job! Your query is correct and returns t...


In [26]:
# save the dataframe with zero-shot mega prompts and GPT responses for evaluation
data1.to_csv('zero_shots_completion_800.csv')

## Mega-Prompt Generation

In [47]:
# A function that generates few-shot mega-prompts for a row given the expected number of submissions
from numpy import row_stack
def generate_fewshot_prompt_for_row(
    test, submission_size: int = None
) -> str:
    """
    Generate a mega prompt for a specific QuestionId.

    Args:
        test (pd.DataFrame): The DataFrame - this row.
        submission_size (int, optional): The number of submissions to include in the mega prompt. If None, include all submissions.

    Returns:
        str: The mega prompt.
    """
    temp = df_all[df_all["QuestionID"] == test['QuestionID']]

    # If there's only one row or less, return None or handle it in a special way
    if len(temp) == 0:
      return None
    if len(temp) == 1:
      # return the only row as test answer
      temp = temp.reset_index()
      submissions = ''
      db = temp.loc[0, "Database"]
      tables = temp.loc[0, "schema"]
      question = temp.loc[0, "Question"]
      key = temp.loc[0, "AnswerKey"]

      test = temp.loc[0, "SubmittedAnswer"]

      mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade (out of 10 points) and feedback for the student
        """
      return mega_p

    # skip the current row
    temp = temp[temp['StudentID']!=test['StudentID']]
    if submission_size==None or submission_size >= temp.shape[0]:
      train_df = temp.copy().reset_index()
    else:
      # randomly sampling certain size of training rows for the same QuestionId
        train_df = temp.sample(n=submission_size).reset_index()

    submissions = generate_submissions_string(train_df)

    db = train_df.loc[0, "Database"]
    tables = train_df.loc[0, "schema"]
    question = train_df.loc[0, "Question"]
    key = train_df.loc[0, "AnswerKey"]
    test = test['SubmittedAnswer']

    mega_p = f"""
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

      Please provide a grade and feedback for the student.

      """ + """Only provide a RFC8259 compliant JSON response following this format without deviation. {"grade": "the grade out of 10 you give", "feedback": "the feedback you give"}"""
    return mega_p


## Sample Records from Original DF

In [48]:
# Randomly select 300 rows from the dataframe
data1, _ = train_test_split(df_all, train_size=400, stratify=df_all['QuestionID'], random_state=1234)
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score
9492,56,"SELECT C.CustomerID, C.CompanyName, COUNT(O.Or...",sgd6820,7.5,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0
11693,9,SELECT * FROM track WHERE artist_id = 3 AND ...,km6355,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
10859,15,SELECT * FROM album AS A \tINNER JOIN artis...,jlt9336,0.0,Assignment 3: Joins,List all the album names by the band New Order,music,"SELECT R.artist_name, A.* FROM artist R JOIN a...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0
232,2,"SELECT album_name AS album_title, artist_id FR...",fam7,0.0,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR...","[{'album': ['artist_id', 'album_id', 'album_na...",11.0
5933,76,I took a picture of my SQL Commands and Output...,dek8225,0.0,Module 2 Practice: Filtering queries,"Find all students with first name ""Richard""",facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'Rich...,"[{'Concentration': ['ProfileID', 'Concentratio...",0.0


In [49]:
# Generate few-shot mega prompts with submission size = all for GPT4-32k model
lst = []
for i,r in data1.iterrows():
  lst.append(generate_fewshot_prompt_for_row(r))

data1['4_few_mp'] = lst
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,4_few_mp
9492,56,"SELECT C.CustomerID, C.CompanyName, COUNT(O.Or...",sgd6820,7.5,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0,\n a. We are working with the northwind d...
11693,9,SELECT * FROM track WHERE artist_id = 3 AND ...,km6355,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...
10859,15,SELECT * FROM album AS A \tINNER JOIN artis...,jlt9336,0.0,Assignment 3: Joins,List all the album names by the band New Order,music,"SELECT R.artist_name, A.* FROM artist R JOIN a...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...
232,2,"SELECT album_name AS album_title, artist_id FR...",fam7,0.0,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR...","[{'album': ['artist_id', 'album_id', 'album_na...",11.0,\n a. We are working with the music datab...
5933,76,I took a picture of my SQL Commands and Output...,dek8225,0.0,Module 2 Practice: Filtering queries,"Find all students with first name ""Richard""",facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'Rich...,"[{'Concentration': ['ProfileID', 'Concentratio...",0.0,\n a. We are working with the facebook da...


In [29]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [50]:
import tiktoken

# counting the number of tokens of mega-prompts
def num_tokens_from_string(string: str, encoding_name="cl100k_base") -> int:
  encoding = tiktoken.get_encoding(encoding_name)
  num_tokens = len(encoding.encode(string))
  return num_tokens

In [51]:
# Counting the number of tokens for each mega prompt
data1['num'] = data1['4_few_mp'].apply(num_tokens_from_string)
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,4_few_mp,num
9492,56,"SELECT C.CustomerID, C.CompanyName, COUNT(O.Or...",sgd6820,7.5,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0,\n a. We are working with the northwind d...,12833
11693,9,SELECT * FROM track WHERE artist_id = 3 AND ...,km6355,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,9830
10859,15,SELECT * FROM album AS A \tINNER JOIN artis...,jlt9336,0.0,Assignment 3: Joins,List all the album names by the band New Order,music,"SELECT R.artist_name, A.* FROM artist R JOIN a...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,12907
232,2,"SELECT album_name AS album_title, artist_id FR...",fam7,0.0,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR...","[{'album': ['artist_id', 'album_id', 'album_na...",11.0,\n a. We are working with the music datab...,9512
5933,76,I took a picture of my SQL Commands and Output...,dek8225,0.0,Module 2 Practice: Filtering queries,"Find all students with first name ""Richard""",facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'Rich...,"[{'Concentration': ['ProfileID', 'Concentratio...",0.0,\n a. We are working with the facebook da...,6946


In [52]:
print(data1['num'].max())

41154


In [33]:
from tqdm import tqdm
import json

In [53]:
# a function that takes the current row and maximum number of tokens we need
# need improvement since we select training rows randomly
def max_submission_size(t, max_size):
  idf = df_all[df_all["QuestionID"] == t['QuestionID']]
  for i in range(1,len(idf)-1):
    mp = generate_fewshot_prompt_for_row(t,i)
    if num_tokens_from_string(mp, "cl100k_base") >= max_size:
      return i-1
  return None

In [56]:
# 8196 tokens limit for gpt4, select a maximum submission size for each row
lst = []
for i, r in tqdm(data1.iterrows(), total=len(data1)):
  s_size = max_submission_size(r, 6500)
  lst.append(generate_fewshot_prompt_for_row(r,s_size))

data1['4_few_mp'] = lst
data1.head()

100%|██████████| 400/400 [04:33<00:00,  1.46it/s]


,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,4_few_mp,num
9492,56,"SELECT C.CustomerID, C.CompanyName, COUNT(O.Or...",sgd6820,7.5,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0,\n a. We are working with the northwind d...,6735
11693,9,SELECT * FROM track WHERE artist_id = 3 AND ...,km6355,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,6503
10859,15,SELECT * FROM album AS A \tINNER JOIN artis...,jlt9336,0.0,Assignment 3: Joins,List all the album names by the band New Order,music,"SELECT R.artist_name, A.* FROM artist R JOIN a...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,6612
232,2,"SELECT album_name AS album_title, artist_id FR...",fam7,0.0,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR...","[{'album': ['artist_id', 'album_id', 'album_na...",11.0,\n a. We are working with the music datab...,6469
5933,76,I took a picture of my SQL Commands and Output...,dek8225,0.0,Module 2 Practice: Filtering queries,"Find all students with first name ""Richard""",facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'Rich...,"[{'Concentration': ['ProfileID', 'Concentratio...",0.0,\n a. We are working with the facebook da...,6518


In [57]:
data1['num'] = data1['4_few_mp'].apply(num_tokens_from_string)
data1.head()
print(data1['num'].max()) # print the maximum number of tokens among all mega prompts, which is < 8196

7345


In [58]:
tqdm.pandas()
# generate GPT4 answer for each mega prompt as a new column of JSON objects
data1['4_answer'] = data1['4_few_mp'].progress_apply(delayed_completion)

100%|██████████| 400/400 [27:15<00:00,  4.09s/it]


In [59]:
# parse grade and feedback from json object
def extract_grade(js):
  try:
    js = json.loads(js)
    return js["grade"]
  except json.JSONDecodeError:
    return ''

def extract_feedback(js):
  try:
    js = json.loads(js)
    return js["feedback"]
  except json.JSONDecodeError:
    return ''

In [60]:
# Extract grade and feedback as new columns
data1['4_grade'] = data1['4_answer'].apply(extract_grade)
data1['4_feedback'] = data1['4_answer'].apply(extract_feedback)
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,4_few_mp,num,4_answer,4_grade,4_feedback
9492,56,"SELECT C.CustomerID, C.CompanyName, COUNT(O.Or...",sgd6820,7.5,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0,\n a. We are working with the northwind d...,6168,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query correctly returns th...
11693,9,SELECT * FROM track WHERE artist_id = 3 AND ...,km6355,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,6677,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query is correct and it su...
10859,15,SELECT * FROM album AS A \tINNER JOIN artis...,jlt9336,0.0,Assignment 3: Joins,List all the album names by the band New Order,music,"SELECT R.artist_name, A.* FROM artist R JOIN a...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,6112,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query correctly retrieves ...
232,2,"SELECT album_name AS album_title, artist_id FR...",fam7,0.0,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR...","[{'album': ['artist_id', 'album_id', 'album_na...",11.0,\n a. We are working with the music datab...,5874,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query correctly selects th...
5933,76,I took a picture of my SQL Commands and Output...,dek8225,0.0,Module 2 Practice: Filtering queries,"Find all students with first name ""Richard""",facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'Rich...,"[{'Concentration': ['ProfileID', 'Concentratio...",0.0,\n a. We are working with the facebook da...,5993,"{""grade"": ""0/10"", ""feedback"": ""Your submission...",0/10,Your submission was incorrect. The task was to...


In [61]:
data1['4_grade'].value_counts()

10/10     256
10         47
0/10       24
7.5/10     14
9.5/10     10
7/10        6
5/10        6
8.5/10      5
9.0         4
5.0         3
8/10        3
0.0         3
6.5/10      2
0.0/10      2
3/10        2
8.0         2
5.0/10      2
9/10        2
3.5/10      1
7.0         1
8.5         1
2.0         1
7.5         1
5.5/10      1
2/10        1
Name: 4_grade, dtype: int64

In [69]:
# Repeat the same process for GPT3.5-turbo mega prompt and responses
lst = []
for i, r in tqdm(data1.iterrows(), total=len(data1)):
  s_size = max_submission_size(r, 2500) # max # of tokens = 4096
  lst.append(generate_fewshot_prompt_for_row(r,s_size))

data1['3_few_mp'] = lst

data1['num'] = data1['3_few_mp'].apply(num_tokens_from_string)
print(data1['num'].max())
data1.head()

100%|██████████| 400/400 [01:04<00:00,  6.20it/s]


3098


,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,4_few_mp,num,4_answer,4_grade,4_feedback,3_few_mp
9492,56,"SELECT C.CustomerID, C.CompanyName, COUNT(O.Or...",sgd6820,7.5,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0,\n a. We are working with the northwind d...,2118,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query correctly returns th...,\n a. We are working with the northwind d...
11693,9,SELECT * FROM track WHERE artist_id = 3 AND ...,km6355,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2160,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query is correct and it su...,\n a. We are working with the music datab...
10859,15,SELECT * FROM album AS A \tINNER JOIN artis...,jlt9336,0.0,Assignment 3: Joins,List all the album names by the band New Order,music,"SELECT R.artist_name, A.* FROM artist R JOIN a...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2363,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query correctly retrieves ...,\n a. We are working with the music datab...
232,2,"SELECT album_name AS album_title, artist_id FR...",fam7,0.0,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR...","[{'album': ['artist_id', 'album_id', 'album_na...",11.0,\n a. We are working with the music datab...,2349,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query correctly selects th...,\n a. We are working with the music datab...
5933,76,I took a picture of my SQL Commands and Output...,dek8225,0.0,Module 2 Practice: Filtering queries,"Find all students with first name ""Richard""",facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'Rich...,"[{'Concentration': ['ProfileID', 'Concentratio...",0.0,\n a. We are working with the facebook da...,2530,"{""grade"": ""0/10"", ""feedback"": ""Your submission...",0/10,Your submission was incorrect. The task was to...,\n a. We are working with the facebook da...


In [70]:
tqdm.pandas()
data1['3_answer'] = data1['3_few_mp'].progress_apply(GPT3_generation_with_backoff)

100%|██████████| 400/400 [12:47<00:00,  1.92s/it]


In [71]:
data1['3_grade'] = data1['3_answer'].apply(extract_grade)
data1['3_feedback'] = data1['3_answer'].apply(extract_feedback)
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,4_few_mp,num,4_answer,4_grade,4_feedback,3_few_mp,3_answer,3_grade,3_feedback
9492,56,"SELECT C.CustomerID, C.CompanyName, COUNT(O.Or...",sgd6820,7.5,Final Exam,For every customer in the database list the nu...,northwind,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","[{'Categories': ['CategoryID', 'CategoryName',...",10.0,\n a. We are working with the northwind d...,2118,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query correctly returns th...,\n a. We are working with the northwind d...,"{""grade"": ""7.5/10.0"", ""feedback"": ""Good job on...",7.5/10.0,Good job on correctly joining the Customers an...
11693,9,SELECT * FROM track WHERE artist_id = 3 AND ...,km6355,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2160,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query is correct and it su...,\n a. We are working with the music datab...,"{""grade"": ""9.5"", ""feedback"": ""Great job! Your ...",9.5,Great job! Your query correctly retrieves the ...
10859,15,SELECT * FROM album AS A \tINNER JOIN artis...,jlt9336,0.0,Assignment 3: Joins,List all the album names by the band New Order,music,"SELECT R.artist_name, A.* FROM artist R JOIN a...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,2363,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query correctly retrieves ...,\n a. We are working with the music datab...,"{""grade"": ""8.5"", ""feedback"": ""Great job! Your ...",8.5,Great job! Your query successfully retrieves t...
232,2,"SELECT album_name AS album_title, artist_id FR...",fam7,0.0,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR...","[{'album': ['artist_id', 'album_id', 'album_na...",11.0,\n a. We are working with the music datab...,2349,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query correctly selects th...,\n a. We are working with the music datab...,"{""grade"": ""0.0/11.0"", ""feedback"": ""The query i...",0.0/11.0,The query is incorrect. The requested columns ...
5933,76,I took a picture of my SQL Commands and Output...,dek8225,0.0,Module 2 Practice: Filtering queries,"Find all students with first name ""Richard""",facebook,SELECT *\nFROM Profiles\nWHERE Name LIKE 'Rich...,"[{'Concentration': ['ProfileID', 'Concentratio...",0.0,\n a. We are working with the facebook da...,2530,"{""grade"": ""0/10"", ""feedback"": ""Your submission...",0/10,Your submission was incorrect. The task was to...,\n a. We are working with the facebook da...,"{""grade"": ""7.5"", ""feedback"": ""The student's an...",7.5,The student's answer is partially correct. The...


In [72]:
data1['3_grade'].value_counts()

8.5         126
7.5         113
6.5          25
9.5          23
10           14
0.9          13
1.0          11
7.5/10.0      9
5.0           8
0.8           7
0.7           6
10.0          5
1.0/11.0      5
7.5/10        5
7.0           5
7             3
2             3
10/10         2
0.5           2
              2
1.5           2
4.5           2
9.5/10.0      1
8             1
9.0           1
6             1
0.0/11.0      1
2.5           1
1.8           1
0.95          1
0.0           1
Name: 3_grade, dtype: int64

In [73]:
data1.to_csv('few_completions_final.csv')